## Context

- https://github.com/JuliaReach/LazySets.jl/issues/682
- https://github.com/JuliaReach/LazySets.jl/pull/793

## WIP code

In [ ]:
function _sandwich_search(ℓ, X, H::Union{HalfSpace, Hyperplane, Line}; kwargs...)
    options = Dict(kwargs)

    # Initialization
    a, b = H.a, H.b
    r⁻, r⁺ = -Inf, Inf

    if haskey(options, :lower)
        lower = pop!(options, :lower)
    else
        if H isa HalfSpace
            lower = 0.0
        elseif (H isa Hyperplane) || (H isa Line)
            lower = -1e6 # "big": TODO relate with f(λ)
        end
    end

    if haskey(options, :upper)
        upper = pop!(options, :upper)
    else
        upper = 1e6 # "big": TODO relate with f(λ)
    end

    if haskey(options, :method)
        method = pop!(options, :method)
    else
        method = Optim.Brent()
    end

    # initialization

    for i in 3:length(λ)
        λ1, λ2, λ3 = λ[i], λ[i-1], λ[i-2]
        f_1(λ) = ρ(ℓ - λ1 * a, X) + λ1 * b
        f_2(λ) = ρ(ℓ - λ2 * a, X) + λ2 * b
        f_3(λ) = ρ(ℓ - λ3 * a, X) + λ3 * b

        while (!(f_1(λ) <= f_2(λ) && f_2(λ) <= f_3(λ)))
            #TODO make exponential growth
        end

        λi, λj = 0, 0 #TODO Generate two samples, such that λi < λj
        f⁻ij(λ) = ((f(λj) - f(λi))/(λj - λi))(λj - λi) + f(λi)

        sol_r⁻ = Optim.optimize(f⁻, lower, upper, method=method, options...)
        fmin_r⁻, λmin_r⁻ = sol_r⁻.minimum, sol_r⁻.minimize
        sol_r⁺ = Optim.optimize(f, lower, upper, method=method, options...)
        fmin_r⁺, λmin_r⁺ = sol_r⁺.minimum, sol_r⁺.minimize
        if (r⁺ - r⁻ < ε)
            return Interval([r⁻, r⁺])
        end
    end